In [1]:
import pandas as pd
import numpy as np
from scipy import stats

In [2]:
df = pd.read_csv('xd.csv')

In [3]:
grouped_df = [group for _, group in df.groupby(['type', 'pages'], as_index=False)]

In [4]:
for group in grouped_df:
    battery = group['battery']
    time = group['time']

    battery_q25, battery_q75 = np.percentile(battery, 25), np.percentile(battery, 75)
    battery_iqr = battery_q75 - battery_q25

    battery_cutoff = battery_iqr * 1.5
    battery_lower, battery_upper = battery_q25 - battery_cutoff, battery_q75 + battery_cutoff


    time_q25, time_q75 = np.percentile(time, 25), np.percentile(time, 75)
    time_iqr = time_q75 - time_q25

    time_cutoff = time_iqr * 1.5
    time_lower, time_upper = time_q25 - time_cutoff, time_q75 + time_cutoff

    isAnomaly = [
        int(
            (row['time'] < time_lower or row['time'] > time_upper)
            or
            (row['battery'] < battery_lower or row['battery'] > battery_upper)
        )
        for _, row
        in group.iterrows()
    ]
    group['isAnomaly'] = isAnomaly

    print(group[group['isAnomaly'] == 1])
    print()

      type   time  size  battery  pages  iterations  isAnomaly
280  cloud  57.37  0.13  -133158      1          10          1
348  cloud  52.37  0.13    -2429      1          10          1
391  cloud  53.64  0.13    -1967      1          10          1
483  cloud  57.33  0.13    -2065      1          10          1
501  cloud  57.36  0.13    -2150      1          10          1
530  cloud  51.87  0.13    -2392      1          10          1
539  cloud  53.26  0.08    -2182      1          10          1
541  cloud  57.98  0.08    -2198      1          10          1
555  cloud  57.76  0.08    -2164      1          10          1
562  cloud  52.69  0.08    -2522      1          10          1
565  cloud  52.89  0.08    -2040      1          10          1
572  cloud  53.19  0.08    -2150      1          10          1
581  cloud  63.10  0.08    -2340      1          10          1
589  cloud  52.35  0.08    -2940      1          10          1
598  cloud  58.82  0.08    -2535      1          10    

In [5]:
labeled_df = pd.concat(grouped_df).sort_index()

In [6]:
labeled_df.to_csv('labeled.csv', index=False)